In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from tpot import TPOTClassifier

In [3]:
train_bike = pd.read_csv("train.csv")

In [4]:
train_bike["datetime"] = pd.to_datetime(train_bike["datetime"])

In [5]:
train_bike["dayofweek"] = train_bike["datetime"].dt.dayofweek      #The day of the week with Monday=0, Sunday=6
train_bike["year"] = train_bike["datetime"].dt.year
train_bike["month"] = train_bike["datetime"].dt.month
train_bike["day"] = train_bike["datetime"].dt.day
train_bike["hour"] = train_bike["datetime"].dt.hour

In [6]:
X = train_bike[["season", "holiday", "workingday", "weather","temp","humidity","windspeed"]]
y = train_bike["registered"]

In [ ]:
from sklearn.model_selection import train_test_split #dzielnie na zetsawy
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, random_state=42)

In [ ]:
best_score = 0
parameters = {}

In [8]:
train_bike.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,dayofweek,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,5,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,5,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,5,2011,1,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,5,2011,1,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,5,2011,1,1,4


In [10]:
train_bike.dtypes

datetime      datetime64[ns]
season                 int64
holiday                int64
workingday             int64
weather                int64
temp                 float64
atemp                float64
humidity               int64
windspeed            float64
casual                 int64
registered             int64
count                  int64
dayofweek              int64
year                   int64
month                  int64
day                    int64
hour                   int64
dtype: object